In [1]:
import pandas as pd
import numpy as np

In [2]:
투자자 = pd.read_csv('투자자20201103.csv', encoding='euc-kr')
투자자.head()

,Unnamed: 0,종목,시가,1,종가1,2,종가2,3,종가3,4,...,프로그램순매수1,프로그램순매수2,프로그램순매수3,프로그램순매수4,프로그램순매수5,프로그램순매수6,프로그램순매수7,프로그램순매수8,프로그램순매수9,프로그램순매수10
0,0,2170,59600,2020-09-08,59600,2020-09-09,59400,2020-09-10,58700,2020-09-11,...,-80.0,177.0,-1568.0,-1345.0,-534.0,948.0,-260.0,-225.0,-516.0,1847.0
1,1,2170,58700,2020-09-11,58300,2020-09-14,59400,2020-09-15,60300,2020-09-16,...,-1345.0,-534.0,948.0,-260.0,-225.0,-516.0,1847.0,-595.0,-462.0,-563.0
2,2,2200,21300,2018-07-16,21600,2018-07-17,21200,2018-07-18,20900,2018-07-19,...,1251.0,-593.0,-1385.0,-1721.0,-486.0,-506.0,-1174.0,-1223.0,-646.0,-360.0
3,3,2200,21700,2018-07-17,21200,2018-07-18,20900,2018-07-19,21800,2018-07-20,...,-593.0,-1385.0,-1721.0,-486.0,-506.0,-1174.0,-1223.0,-646.0,-360.0,-366.0
4,4,2200,21500,2018-07-20,21200,2018-07-23,21800,2018-07-24,20900,2018-07-25,...,-486.0,-506.0,-1174.0,-1223.0,-646.0,-360.0,-366.0,-63.0,46.0,-1646.0


In [3]:
#s_type = 투자자 유형
#s_date = 매수날짜 

def get_순매수(s_type,s_date):
    순매수 = 투자자[
    (
                ((투자자.filter(regex = '^{}순매수1$'.format(s_type))['{}순매수1'.format(s_type)]>0) & 
                 (투자자.filter(regex = '^{}순매수2$'.format(s_type))['{}순매수2'.format(s_type)]>0))
                 |
                ((투자자.filter(regex = '^{}순매수2$'.format(s_type))['{}순매수2'.format(s_type)]>0) & 
                 (투자자.filter(regex = '^{}순매수3$'.format(s_type))['{}순매수3'.format(s_type)]>0))
                 |
                ((투자자.filter(regex = '^{}순매수1$'.format(s_type))['{}순매수1'.format(s_type)]>0) & 
                 (투자자.filter(regex = '^{}순매수3$'.format(s_type))['{}순매수3'.format(s_type)]>0))
                 |
            ((투자자.filter(regex = '^{}순매수1$'.format(s_type))['{}순매수1'.format(s_type)]>0) &
             (투자자.filter(regex = '^{}순매수2$'.format(s_type))['{}순매수2'.format(s_type)]>0) & 
             (투자자.filter(regex = '^{}순매수3$'.format(s_type))['{}순매수3'.format(s_type)]>0))
    )].copy()
    
    #투자자 유형과 종목코드 설정 
    순매수.loc[:,'등락률_유형'] = s_type
    순매수.rename(columns = {'종목':'종목_등락률'},inplace = True)
    
    #등락률 계산 
    for i in range(10-s_date):
        순매수['등락률'+ str(s_date)+'매수' + str(10-i) +'매도'] = (순매수['종가'+ str(10-i)] - 순매수['종가' + str(s_date)]) / 순매수['종가'+ str(s_date)]
    
    #종목별로 groupby
    등락률 = 순매수.filter(like  = '등락률',axis= 1)
    종목별_평균 = 등락률.groupby('종목_등락률',as_index = False).mean()
    종목별_평균['투자자 유형'] = 등락률.iloc[0]['등락률_유형']

    #컬럼 이름을 다시 원래대로 설정 
    종목별_평균.rename(columns = {'종목_등락률':'종목 코드'},inplace = True)

    #투자자 유형과 종목 코드를 인덱스로 설정 
    종목별_평균 = 종목별_평균.set_index(['투자자 유형','종목 코드'])
    종목별_평균
    return 종목별_평균

In [4]:
#반복문을 이용해주기위해 
투자자_리스트 = ['개인','기관','등록외국','비등록외국','외국계','외국','프로그램']

#반복문안에서 리턴되는 데이터프레임 병합용
final_등락률 = pd.DataFrame()

for each in 투자자_리스트:
    final_등락률 = final_등락률.append(get_순매수(each,3))

In [6]:
final_등락률

등락률3매수10매도  등락률3매수9매도  등락률3매수8매도  등락률3매수7매도  등락률3매수6매도  \
투자자 유형 종목 코드                                                            
개인     20        0.008923   0.007665   0.006844   0.003644   0.001498   
       40        0.000095  -0.001896  -0.004594  -0.003730  -0.003761   
       50       -0.003691  -0.002862  -0.001774  -0.002074  -0.001006   
       60        0.002044   0.002387   0.002852   0.002057   0.001452   
       70       -0.004262  -0.003239  -0.002075  -0.001082  -0.000206   
...                   ...        ...        ...        ...        ...   
프로그램   355150   -0.006194  -0.004954  -0.003716  -0.001235   0.001241   
       357120    0.006999   0.005949   0.005778   0.005248   0.004194   
       357250    0.024419   0.016399   0.010110   0.004556   0.002127   
       357780   -0.029657  -0.042218  -0.049008  -0.036195  -0.026285   
       363280   -0.128444  -0.121226  -0.104794  -0.110616  -0.066437   

               등락률3매수5매도  등락률3매수4매도  
투자자 유형 종목 코드                         
개인     20      -0.000380  -0.001019  
       40      -0.004513  -0.004138  
       50      -0.000327  -0.000729  
       60       0.000680  -0.000700  
       70      -0.000675  -0.000909  
...                  ...        ...  
프로그램   355150   0.001241   0.001241  
       357120   0.002269   0.000524  
       357250   0.001074   0.002811  
       357780  -0.013531  -0.014331  
       363280  -0.069929  -0.040245  

[15848 rows x 7 columns]

In [7]:
for i in range(7):
    print(final_등락률.loc[final_등락률['등락률3매수{}매도'.format(10-i)].idxmax(),['등락률3매수{}매도'.format(10-i)]])

등락률3매수10매도    0.51
Name: (비등록외국, 279060), dtype: float64
등락률3매수9매도    0.614542
Name: (비등록외국, 227420), dtype: float64
등락률3매수8매도    0.631474
Name: (비등록외국, 227420), dtype: float64
등락률3매수7매도    0.517928
Name: (비등록외국, 227420), dtype: float64
등락률3매수6매도    0.419323
Name: (비등록외국, 227420), dtype: float64
등락률3매수5매도    0.25
Name: (비등록외국, 279060), dtype: float64
등락률3매수4매도    0.15
Name: (비등록외국, 279060), dtype: float64


In [18]:
final_등락률.to_csv("all.csv", encoding='euc-kr')

##### 종목관계없이

In [8]:
def get_avg_순매수(s_type,s_date):
    순매수 = 투자자[
    (
                ((투자자.filter(regex = '^{}순매수1$'.format(s_type))['{}순매수1'.format(s_type)]>0) & 
                 (투자자.filter(regex = '^{}순매수2$'.format(s_type))['{}순매수2'.format(s_type)]>0))
                 |
                ((투자자.filter(regex = '^{}순매수2$'.format(s_type))['{}순매수2'.format(s_type)]>0) & 
                 (투자자.filter(regex = '^{}순매수3$'.format(s_type))['{}순매수3'.format(s_type)]>0))
                 |
                ((투자자.filter(regex = '^{}순매수1$'.format(s_type))['{}순매수1'.format(s_type)]>0) & 
                 (투자자.filter(regex = '^{}순매수3$'.format(s_type))['{}순매수3'.format(s_type)]>0))
                 |
            ((투자자.filter(regex = '^{}순매수1$'.format(s_type))['{}순매수1'.format(s_type)]>0) &
             (투자자.filter(regex = '^{}순매수2$'.format(s_type))['{}순매수2'.format(s_type)]>0) & 
             (투자자.filter(regex = '^{}순매수3$'.format(s_type))['{}순매수3'.format(s_type)]>0))
    )].copy()
    
    #투자자 유형과 종목코드 설정 
    순매수.loc[:,'등락률_유형'] = s_type
    순매수.rename(columns = {'종목':'종목_등락률'},inplace = True)
    
    #등락률 계산 
    for i in range(10-s_date):
        순매수['등락률'+ str(s_date)+'매수' + str(10-i) +'매도'] = (순매수['종가'+ str(10-i)] - 순매수['종가' + str(s_date)]) / 순매수['종가'+ str(s_date)]
    
    average = pd.DataFrame(columns = ['매수/매도','평균 등락률'])
    
    등락률 = 순매수.filter(like  = '등락률',axis= 1)
    등락률 = 등락률.filter(regex = '매도$')

    for each in 등락률.columns:
        average = average.append(pd.DataFrame([[each,등락률[each].mean()]],columns = ['매수/매도','평균 등락률']),ignore_index = True)
    average['투자자유형'] = s_type
    return average

In [10]:
투자자_리스트 = ['개인','기관','등록외국','비등록외국','외국계','외국','프로그램']

avg_등락률 = pd.DataFrame()

for each in 투자자_리스트:
    avg_등락률 = avg_등락률.append(get_avg_순매수(each,3))
    
avg_등락률 = avg_등락률.set_index('투자자유형')
avg_등락률.sort_values(by = ['평균 등락률'],ascending = False).head()

,매수/매도,평균 등락률
투자자유형,,
등록외국,등락률3매수10매도,0.006906
외국,등락률3매수10매도,0.006889
외국계,등락률3매수10매도,0.006506
프로그램,등락률3매수10매도,0.006390
등록외국,등락률3매수9매도,0.006083
